# KV-Cache

1. **计算效率**：只计算新token的Q，重复使用之前token的K和V
2. **内存效率**：缓存K和V避免了重复计算
3. **生成过程**：每次只需要处理当前token，但能获得完整序列的上下文信息

1. **KV-Cache目的**：优化自回归生成过程的计算效率，避免对已生成token的重复计算
2. **实现方式**：缓存每个transformer层的K和V矩阵，在生成新token时只需计算当前token的Q
3. **效果对比**：
   - 无KV-Cache：每次生成需处理整个序列，计算量随序列长度线性增长
   - 有KV-Cache：每次生成只需处理当前token，计算量基本恒定


In [1]:
import torch
import torch.nn.functional as F
from transformers import LlamaModel, LlamaConfig, LlamaForCausalLM
import time

# 配置一个小型LLaMA模型
config = LlamaConfig(
    vocab_size=100,
    hidden_size=256,
    intermediate_size=512,
    num_hidden_layers=2,
    num_attention_heads=4,
    num_key_value_heads=4,
)
model = LlamaForCausalLM(config)  # 加载模型

# 创建输入数据 (batch_size=1, seq_len=10)
X = torch.randint(0, 100, (1, 10))  # 直接使用随机整数代替tokenizer

# 自回归生成过程
idx = {'input_ids': X}
for i in range(4):
    print(f"\nGeneration第{i}个时的输入 shape: {idx['input_ids'].shape}")
    output = model(**idx)
    logits = output['logits'][:, -1, :]  # 只取最后一个token的logits
    idx_next = torch.argmax(logits, dim=1)[0]  # 贪心搜索
    
    # 将新生成的token拼接到输入中
    idx['input_ids'] = torch.cat((idx['input_ids'], idx_next.unsqueeze(0).unsqueeze(1)), dim=-1)
    time.sleep(1)  # 模拟实际生成延迟


Generation第0个时的输入 shape: torch.Size([1, 10])

Generation第1个时的输入 shape: torch.Size([1, 11])

Generation第2个时的输入 shape: torch.Size([1, 12])

Generation第3个时的输入 shape: torch.Size([1, 13])


In [2]:
class DecoderWithKVCache(torch.nn.Module):
    def __init__(self, D, V):
        super().__init__()
        self.D = D  # 单头注意力维度
        self.V = V  # 词表大小
        self.Embedding = torch.nn.Embedding(V, D)
        self.Wq = torch.nn.Linear(D, D)  # Q矩阵
        self.Wk = torch.nn.Linear(D, D)  # K矩阵
        self.Wv = torch.nn.Linear(D, D)  # V矩阵
        self.lm_head = torch.nn.Linear(D, V)  # 语言模型头
        self.cache_K = self.cache_V = None  # KV缓存初始化

    def forward(self, X):
        X = self.Embedding(X)
        Q, K, V = self.Wq(X), self.Wk(X), self.Wv(X)
        print(f"input_Q: {Q.shape}")
        print(f"input_K: {K.shape}")
        print(f"input_V: {V.shape}")

        # KV-Cache机制
        if self.cache_K is None:  # 第一次生成
            self.cache_K = K
            self.cache_V = V
        else:  # 后续生成
            self.cache_K = torch.cat((self.cache_K, K), dim=1)  # 拼接新K值
            self.cache_V = torch.cat((self.cache_V, V), dim=1)  # 拼接新V值
            K, V = self.cache_K, self.cache_V  # 使用完整缓存

        print(f"cache_K: {self.cache_K.shape}")
        print(f"cache_V: {self.cache_V.shape}")

        # 简化版注意力计算(实际应用中会有缩放、多头等处理)
        attn = Q @ K.transpose(1, 2) @ V
        return self.lm_head(attn)

# 使用示例
model = DecoderWithKVCache(D=128, V=64)
# 创建数据、不使用tokenizer
X = torch.randint(0, 64, (1, 10))  # 初始输入

print(X.shape)

for i in range(3):
    print(f"\nGeneration {i} step input_shape: {X.shape}")
    output = model.forward(X)
    next_token = torch.argmax(F.softmax(output, dim=-1), -1)[:, -1]
    print(f'next_token预测: {next_token}')
    # 注意这里 X 取每次新生成的 next token，而不是和之前的 input 拼接
    X = next_token.unsqueeze(0)

torch.Size([1, 10])

Generation 0 step input_shape: torch.Size([1, 10])
input_Q: torch.Size([1, 10, 128])
input_K: torch.Size([1, 10, 128])
input_V: torch.Size([1, 10, 128])
cache_K: torch.Size([1, 10, 128])
cache_V: torch.Size([1, 10, 128])
next_token预测: tensor([48])

Generation 1 step input_shape: torch.Size([1, 1])
input_Q: torch.Size([1, 1, 128])
input_K: torch.Size([1, 1, 128])
input_V: torch.Size([1, 1, 128])
cache_K: torch.Size([1, 11, 128])
cache_V: torch.Size([1, 11, 128])
next_token预测: tensor([0])

Generation 2 step input_shape: torch.Size([1, 1])
input_Q: torch.Size([1, 1, 128])
input_K: torch.Size([1, 1, 128])
input_V: torch.Size([1, 1, 128])
cache_K: torch.Size([1, 12, 128])
cache_V: torch.Size([1, 12, 128])
next_token预测: tensor([40])
